# Guided Project: Exploring Hacker News Posts

The project is designed to work and analyze data set of submissions to the technology site Hacker News. The data we will work with will be user-submitted stories (posts), which are being voted and commented upoo. We will be given a sample of the data set consisting of approximeately 20,000 rows, after they were cleaned from posts without any comments.
Our main goal will be to analyze what time the most popular posts are created and do Ask HN or Show HN receive more comments in general.



In [14]:
from csv import reader
opened_file = open("hacker_news.csv")
read_file = reader(opened_file)
hn = list(read_file)
headers = hn[0]
hn = hn[1:]

print(headers)
print(hn[:5])

['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at']
[['12224879', 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', '386', '52', 'ne0phyte', '8/4/2016 11:52'], ['10975351', 'How to Use Open Source and Shut the Fuck Up at the Same Time', 'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/', '39', '10', 'josep2', '1/26/2016 19:30'], ['11964716', "Florida DJs May Face Felony for April Fools' Water Joke", 'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/', '2', '1', 'vezycash', '6/23/2016 22:20'], ['11919867', 'Technology ventures: From Idea to Enterprise', 'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429', '3', '1', 'hswarna', '6/17/2016 0:01'], ['10301696', 'Note by Note: The Making of Steinway L1037 (2007)', 'http://www.nytimes.com/2007/11/07/movies/07stein.html?_r=0', '8', '2', 'walterbell', '9/30/2015 4:12']]


In [15]:
ask_posts = []
show_posts = []
other_posts = []

for row in hn:
    title = row[1]
    if title.lower().startswith('ask hn'):
        ask_posts.append(row)
    elif title.lower().startswith('show hn'):
        show_posts.append(row)
    else:
        other_posts.append(row)
    
    
print(len(ask_posts))
print(len(show_posts))
print(len(other_posts))
print('\n')
print('ask_posts', ask_posts[:5])
print('\n')
print('show_posts', show_posts[:5])
print('\n')
print('other_posts', other_posts[:5])


  

1744
1162
17194


ask_posts [['12296411', 'Ask HN: How to improve my personal website?', '', '2', '6', 'ahmedbaracat', '8/16/2016 9:55'], ['10610020', 'Ask HN: Am I the only one outraged by Twitter shutting down share counts?', '', '28', '29', 'tkfx', '11/22/2015 13:43'], ['11610310', 'Ask HN: Aby recent changes to CSS that broke mobile?', '', '1', '1', 'polskibus', '5/2/2016 10:14'], ['12210105', 'Ask HN: Looking for Employee #3 How do I do it?', '', '1', '3', 'sph130', '8/2/2016 14:20'], ['10394168', 'Ask HN: Someone offered to buy my browser extension from me. What now?', '', '28', '17', 'roykolak', '10/15/2015 16:38']]


show_posts [['10627194', 'Show HN: Wio Link  ESP8266 Based Web of Things Hardware Development Platform', 'https://iot.seeed.cc', '26', '22', 'kfihihc', '11/25/2015 14:03'], ['10646440', 'Show HN: Something pointless I made', 'http://dn.ht/picklecat/', '747', '102', 'dhotson', '11/29/2015 22:46'], ['11590768', 'Show HN: Shanhu.io, a programming playground powered by

In [16]:
total_ask_comments = 0

for row in ask_posts:
    num_comments = int(row[4])
    total_ask_comments += num_comments
avg_ask_comments = total_ask_comments/len(ask_posts)
print(avg_ask_comments)
print(total_ask_comments)
print('\n')

total_show_comments = 0

for row in show_posts:
    num_comments = int(row[4])
    total_show_comments += num_comments
avg_show_comments = total_show_comments/len(show_posts)
print(avg_show_comments)
print(total_show_comments)

    

14.038417431192661
24483


10.31669535283993
11988


*Based on our calculations above, ask posts receive more comments on average.*

In [29]:
import datetime as dt

result_list = []

for row in ask_posts:
    new_list=[]
    created_at  = row[6]
    new_list.append(created_at)
    num_comments = int(row[4])
    new_list.append(num_comments)
    
    result_list.append(new_list)
    
import datetime as dt

result_list = []

for row in ask_posts:
    new_list=[]
    created_at  = row[6]
    new_list.append(created_at)
    num_comments = int(row[4])
    new_list.append(num_comments)
    
    result_list.append(new_list)
      
counts_by_hour = {}
comments_by_hour = {}

for row in result_list:
    hour_str = row[0]
    comment = row[1]
    hour_dt = dt.datetime.strptime(hour_str, "%m/%d/%Y %H:%M")
    hour = hour_dt.strftime("%H")
    
    if hour not in counts_by_hour:
        counts_by_hour[hour] = 1
        comments_by_hour[hour] = comment
    if hour in counts_by_hour:
        counts_by_hour[hour] += 1
        comments_by_hour[hour] += comment
        
print(counts_by_hour)
print(comments_by_hour)

        
    

{'20': 81, '08': 49, '05': 47, '01': 61, '15': 117, '13': 86, '21': 110, '04': 48, '22': 72, '12': 74, '03': 55, '11': 59, '19': 111, '18': 110, '17': 101, '23': 69, '10': 60, '14': 108, '16': 109, '06': 45, '02': 59, '07': 35, '00': 56, '09': 46}
{'20': 1724, '08': 497, '05': 493, '01': 716, '15': 4478, '13': 1282, '21': 1749, '04': 340, '22': 481, '12': 691, '03': 422, '11': 643, '19': 1191, '18': 1441, '17': 1147, '23': 544, '10': 794, '14': 1419, '16': 1831, '06': 398, '02': 1384, '07': 269, '00': 457, '09': 257}


In [27]:
avg_by_hour = []

for hours in comments_by_hour:
    avg_by_hour.append([hours, (comments_by_hour[hours]/counts_by_hour[hours])])
print(avg_by_hour)

[['20', 21.28395061728395], ['08', 10.142857142857142], ['05', 10.48936170212766], ['01', 11.737704918032787], ['15', 38.27350427350427], ['13', 14.906976744186046], ['21', 15.9], ['04', 7.083333333333333], ['22', 6.680555555555555], ['12', 9.337837837837839], ['03', 7.672727272727273], ['11', 10.898305084745763], ['19', 10.72972972972973], ['18', 13.1], ['17', 11.356435643564357], ['23', 7.884057971014493], ['10', 13.233333333333333], ['14', 13.13888888888889], ['16', 16.798165137614678], ['06', 8.844444444444445], ['02', 23.45762711864407], ['07', 7.685714285714286], ['00', 8.160714285714286], ['09', 5.586956521739131]]


In [43]:
swap_avg_by_hour = []
for row in avg_by_hour:
    swap_avg_by_hour.append([row[1], row[0]])
print(swap_avg_by_hour)

sorted_swap = sorted(swap_avg_by_hour, reverse = True)

print("Top 5 Hours for Ask Posts Comments")

for row in sorted_swap[:5]:
    average = row[0]
    hour = row[1]
    hour_strp = dt.datetime.strptime(hour, "%H")
    hour_strf = hour_strp.strftime("%H:%M")
    string = "{}: {:.2f} average comments per post".format(hour_strf,average)
    print(string)

[[21.28395061728395, '20'], [10.142857142857142, '08'], [10.48936170212766, '05'], [11.737704918032787, '01'], [38.27350427350427, '15'], [14.906976744186046, '13'], [15.9, '21'], [7.083333333333333, '04'], [6.680555555555555, '22'], [9.337837837837839, '12'], [7.672727272727273, '03'], [10.898305084745763, '11'], [10.72972972972973, '19'], [13.1, '18'], [11.356435643564357, '17'], [7.884057971014493, '23'], [13.233333333333333, '10'], [13.13888888888889, '14'], [16.798165137614678, '16'], [8.844444444444445, '06'], [23.45762711864407, '02'], [7.685714285714286, '07'], [8.160714285714286, '00'], [5.586956521739131, '09']]
Top 5 Hours for Ask Posts Comments
15:00: 38.27 average comments per post
02:00: 23.46 average comments per post
20:00: 21.28 average comments per post
16:00: 16.80 average comments per post
21:00: 15.90 average comments per post
